In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

# Imports

In [35]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time
import seaborn as sns
import wave, os, glob
import pandas as pd
import numpy as np
from scipy.io import wavfile
from scipy.signal import find_peaks
from scipy.stats import kurtosis, skew

In [ ]:
###On importe chaque .wav pour le transformer en dataframe puis en indiquant son origine via la variable training
zero_a = []
zero_a = pd.DataFrame(zero_a)
path = '\\Python\\Projet\\training\\training-a'
for filename in glob.glob(os.path.join(path, '*.wav')):
    fs, data = wavfile.read(filename)
    df = pd.DataFrame(data)
    df = df.transpose()
    zero_a = zero_a.append(df)
    del data, fs

ref = pd.read_csv('\\REFERENCE.csv',sep = ',')
#On merge selon l'index car l'ordre est le même
base = pd.merge(df, ref, left_index=True, right_index=True)

#Export de la base
base.to_csv('training-a.csv', index  = False)

In [3]:
dataset = pd.read_csv('training-a.csv')

In [4]:
labels = pd.read_csv('training-a/REFERENCE.csv',header=None,names=['filename','label'])['label']
raw_data = dataset.drop(['label'],axis=1)

In [2]:
##ligne perso
dataset = pd.read_csv('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\temp\\training_a.csv')
labels = pd.read_csv('C:\\Users\\benoit\\Desktop\\ENS\\ENSAE\\2A\\S1\\Python\\Projet\\training\\training-a/REFERENCE.csv',header=None,names=['filename','label'])['label']
#raw_data = dataset.drop(['label'],axis=1)
raw_data = dataset.drop(['training'],axis=1)

## Traitement

In [28]:
phases_list = ['S1','S2','systole','diastole']
features_list = ['t1','t2','t12','t21']
phases_peak = ["S1","S2"]
fea_peak = ["S1","S2"]

In [33]:
class PCG:
    
    def __init__(self,signal):
        self.signal = np.asarray(signal[~np.isnan(signal)])-np.mean(signal) #on ignore les NaN et on recentre le signal
        self.fs = 2000
        
    def envelogram(self):
        E = self.signal
        #on remplace les 0 par des petites valeurs (on doit éviter d'avoir un 0 pour calculer les log)
        for i in range(self.signal.shape[0]):
            if E[i]==0:
                E[i] = E[i-1]/10
        E = E/np.max(np.abs(E))
        #E = E/np.std(E)
        E = -1*(E**2) * np.log(E**2)

        #on met les petites valeurs à 0 
        # il faut d'abord calculer les maximums locaux 
        window_length = 750
        local_max = np.array([np.max(E[i-window_length:i+window_length]) for i in range(window_length,len(E)-window_length)])
        local_max = np.concatenate((local_max[0]*np.ones(window_length),local_max,local_max[-1]*np.ones(window_length)))
        s = np.std(E)
        m = np.mean(E)
        mask = E>local_max/5
        mask2 = np.abs(E-m)>s/2
        mask3 = E>np.mean(E)*2
        return E*mask
    
    
    def consec_count(self):
        e = self.envelogram()
        start = 0
        count_mask = []
        while  start < len(e): 
            i = start
            current = e[start]
            while e[i]==current and i < len(e)-1:
                i=i+1
            l = int(i - start + 1)
            count_mask = np.concatenate((count_mask,l*[l]))
            start = i + 1
        return count_mask
    
    def segmentation(self):
        E = self.envelogram()
        count_vec = self.consec_count() # le vecteur de comptage associé
        length_threshold = 50
        beat_mask = count_vec<=length_threshold
        systole_mask = count_vec>length_threshold
        segmentation = np.asarray([])
        mean_beat_length = 0
        beat_counter = 0
        i = 0
        splits = []
        while i<count_vec.shape[0]:
            stop = i 
            next_stop = i
            while stop < count_vec.shape[0] and next_stop-stop<0.100*self.fs:
                stop = next_stop
                #on calcule la longueur du pic
                while stop<count_vec.shape[0] and beat_mask[stop]:
                    stop+=1

                #on calcule l'écart avec le pic suivant, pour voir s'il s'agit du même pic
                next_stop = stop
                while next_stop<count_vec.shape[0] and not beat_mask[next_stop]:
                    next_stop +=1
                
            #un battement long est un battement S1
            if stop-i>mean_beat_length:
                segmentation = np.concatenate((segmentation,np.repeat('S1',stop-i)))
                current_seq = {'beat':'S1','passive':'systole'}
                
            #un battement court est un battement S2
            else:
                segmentation = np.concatenate((segmentation,np.repeat('S2',stop-i)))
                current_seq = {'beat':'S2','passive':'diastole'}
                
            #update beat length and parser
            mean_beat_length = np.average([stop-i,mean_beat_length],weights=[1,beat_counter])
            beat_counter += 1
            i = stop
                
            while stop<count_vec.shape[0] and systole_mask[stop]:
                stop+=1
                
            segmentation = np.concatenate((segmentation,np.repeat(current_seq['passive'],stop-i)))
            #update parser
            i = stop
        
        #on supprime tout ce qui est avant la première phase S1: 
        j = 0 
        while segmentation[j]!='S1':
            j += 1
            
        return segmentation[j:],E[j:],j
    
    
    def timeLengths(self,segmentation,phase):
        res=[]
        start=0
        while start<segmentation.shape[0]:
            i = start
            while i<segmentation.shape[0] and segmentation[i]==phase:
                i+=1
            res.append(i - start)
            start = i + 1
        res = np.array(res)
        return np.array(res[res!=0])
    
    def beatLengths(self,segmentation):
        res = []
        start = 0
        bound = len(segmentation)
        #on commence sur la première phase S1
        while segmentation[start]!='S1':
            start+=1
        #on parcourt toute la segmentation
        while start<bound:
            explorer = start
            while explorer < bound and segmentation[explorer]=='S1':#on finit la phase S1
                explorer += 1
            while explorer < bound and segmentation[explorer]!='S1': #on va jusqu'à la phase S1 suivante
                explorer+=1
            res.append(explorer-start)
            start = explorer
        return np.array(res)
    
    def extractFeatures(self):
        segmentation, E, reste = self.segmentation()
        features = np.array([])
        beat_lengths = self.beatLengths(segmentation)
        #time features
        for phase in phases_list:
            time_features = self.timeLengths(segmentation,phase)
            if phase=='systole' or phase=='diastole':
                if time_features.shape==beat_lengths.shape:
                    ratio = time_features/beatLengths
                else: 
                    time_features = time_features[:-beat_lengths.shape[0]] #on resynchronise, normalement au + 1d'écart
            features = np.append(features,[np.mean(time_features),np.std(time_features)])
        
        
            
        #peak features    
        p_S1 = []
        p_S2 = []
        borne = []
        
        for i in range(len(segmentation)-1):
            if segmentation[i-1] != segmentation[i]:
                borne.append(i)
        for i in range(0,len(borne)-1):
            if segmentation[borne[i]] == 'S1':
                peaks, _ = find_peaks(E[borne[i]:borne[i+1]],height=0,distance = borne[i+1])
                if len(peaks) != 0:
                    index = borne[i] + peaks[0]
                    p_S1.append(E[index])    
            if segmentation[borne[i]] == 'S2':
                peaks, _ = find_peaks(E[borne[i]:borne[i+1]],height=0,distance = borne[i+1])
                if len(peaks) != 0:
                    index = borne[i] + peaks[0]
                    p_S2.append(E[index])

        features = np.append(features,[np.mean(p_S1),np.min(p_S1),np.max(p_S1),np.std(p_S1)])
        features = np.append(features,[np.mean(p_S2),np.min(p_S2),np.max(p_S2),np.std(p_S2)])
        
        #Skewness et Kurtosis
        skew_S1 = []
        skew_S2 = []
        skew_sys = []
        skew_dis = []
        kur_S1 = []
        kur_S2 = []
        kur_sys = []
        kur_dis = []
        
        for i in range(0,len(borne)-1):
            if segmentation[borne[i]] == 'S1':
                skew1 = skew(E[borne[i]:borne[i+1]])
                skew_S1.append(skew1)    
            if segmentation[borne[i]] == 'S2':
                skew2 = skew(E[borne[i]:borne[i+1]])
                skew_S2.append(skew2) 
            if segmentation[borne[i]] == 'diastole':
                skewdis = skew(E[borne[i]:borne[i+1]])
                skew_dis.append(skew_dis) 
            if segmentation[borne[i]] == 'systole':
                skewsys = skew(E[borne[i]:borne[i+1]])
                skew_sys.append(skewsys) 

        for i in range(0,len(borne)-1):
            if segmentation[borne[i]] == 'S1':
                kur1 = kurtosis(E[borne[i]:borne[i+1]])
                kur_S1.append(kur1)    
            if segmentation[borne[i]] == 'S2':
                kur2 = kurtosis(E[borne[i]:borne[i+1]])
                kur_S2.append(kur2) 
            if segmentation[borne[i]] == 'diastole':
                kurdis = kurtosis(E[borne[i]:borne[i+1]])
                kur_dis.append(kurdis) 
            if segmentation[borne[i]] == 'systole':
                kursys = kurtosis(E[borne[i]:borne[i+1]])
                kur_sys.append(kursys) 

        features = np.append(features,[np.mean(skew_S1),np.min(skew_S1),np.max(skew_S1),np.std(skew_S1)])
        features = np.append(features,[np.mean(skew_S2),np.min(skew_S2),np.max(kur_S2),np.std(skew_S2)])
        features = np.append(features,[np.mean(0),np.min(0),np.max(0),np.std(0)])
        features = np.append(features,[np.mean(0),np.min(0),np.max(0),np.std(0)])
        
        features = np.append(features,[np.mean(kur_S1),np.min(kur_S1),np.max(kur_S1),np.std(kur_S1)])
        features = np.append(features,[np.mean(kur_S2),np.min(kur_S2),np.max(kur_S2),np.std(kur_S2)])
        features = np.append(features,[np.mean(0),np.min(0),np.max(0),np.std(0)])
        features = np.append(features,[np.mean(0),np.min(0),np.max(0),np.std(0)])
        
        #mean12 feature
        features = np.append(features,max(features[8],features[12]))
        
        return features

# Visualisation

In [34]:
time_length = 10000
sample = raw_data.loc[0][:time_length]

In [31]:
pcg = PCG(sample)
segmentation = pcg.segmentation()[0]
segmentation

array(['S1', 'S1', 'S1', ..., 'systole', 'systole', 'systole'],
      dtype='<U32')

In [ ]:
print( 'excess kurtosis of normal distribution (should be 0): {}'.format( kurtosis(E) ))
print( 'skewness of normal distribution (should be 0): {}'.format( skew(E) ))

In [32]:
plt.figure(figsize=(14,5))
E,lag = pcg.setStart(pcg.envelogram())
l = time_length-lag
x = np.linspace(0,l/2000,l)
plt.fill_between(x,0,1,where = segmentation=='S1',facecolor='r',alpha=.3,label='S1')
plt.fill_between(x,0,1,where = segmentation=='S2',facecolor='r',alpha=.1,label='S2')
plt.fill_between(x,0,1,where = segmentation=='systole',facecolor='b',alpha=.3,label='systole')
plt.fill_between(x,0,1,where = segmentation=='diastole',facecolor='b',alpha=.1,label='diastole')

plt.plot(x,E/np.max(E),'k',label='signal')
plt.legend()
plt.xlabel('temps (s)')
plt.ylabel('amplitude normalisée')

AttributeError: 'PCG' object has no attribute 'setStart'

# Analyse des features

In [9]:
n_samples = raw_data.shape[0]

In [ ]:
t1,t2,t12,t21=[],[],[],[]
for i in range(n_samples):
    try:
        pcg = PCG(raw_data.loc[i])
        ft = pcg.extractFeatures()
        t1.append(ft['t1'][0])
        t2.append(ft['t2'][0])
        t12.append(ft['t12'][0])
        t21.append(ft['t21'][0])
    except: 
        print(i)
        pass

0
1
2
3
4
5


In [21]:
#Convertion en Df au cas où
X = np.column_stack((t1,t2,t12,t21,m12,ps1,ps2,good))
X_df = pd.DataFrame(X, columns=["t1","t2","t12","t21","mean12","S1","S2","Normal"])
X_df["Normal"] = X_df["Normal"].map({0 : "anormal", 1 : "normal"})

In [22]:
#taleau descriptif
X_df.groupby(["Normal"]).describe().unstack()

            Normal 
S1   count  anormal     29.000000
            normal      71.000000
     mean   anormal      0.294212
            normal       0.267189
     std    anormal      0.068271
            normal       0.079283
     min    anormal      0.095483
            normal       0.033162
     25%    anormal      0.249395
            normal       0.241749
     50%    anormal      0.317319
            normal       0.283967
     75%    anormal      0.348103
            normal       0.322913
     max    anormal      0.365195
            normal       0.367295
S2   count  anormal     29.000000
            normal      71.000000
     mean   anormal      0.159638
            normal       0.134995
     std    anormal      0.048082
            normal       0.053529
     min    anormal      0.094925
            normal       0.016176
     25%    anormal      0.121570
            normal       0.096942
     50%    anormal      0.146255
            normal       0.130643
     75%    anormal      0.1

In [23]:
#Matrice de corrélation
sns.heatmap(X_df.corr())

In [24]:
#Violin plot per variable
features_list = X_df.columns[:-1]
for i in features_list:
    fig = plt.figure()
    fig = plt.figure(figsize=(5,5))
    ax = sns.violinplot(x = [i]*len(X_df), y=X_df[i],hue=X_df["Normal"], palette="Set2", split = True)

C:\Users\benoit\Anaconda3\lib\site-packages\seaborn\categorical.py:637: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  kde_data = remove_na(group_data[hue_mask])
C:\Users\benoit\Anaconda3\lib\site-packages\seaborn\categorical.py:885: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  violin_data = remove_na(group_data[hue_mask])
C:\Users\benoit\Anaconda3\lib\site-packages\seaborn\categorical.py:905: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  violin_data = remove_na(group_data)


In [25]:
good = labels[:n_samples]==1
bad = labels[:n_samples]==-1

In [26]:
#Histogrammes par variable
fig = plt.figure(figsize=(14,10))
ax1 = fig.add_subplot(221)
#t = t
ax1.hist(t1[good],normed=True,color='b',label='normal',alpha=.5)
t = t1[bad]
ax1.hist(t1[bad],normed=True,color='r',label='anormal',alpha=.5)
ax1.set_title('t1')

ax2 = fig.add_subplot(222)
t = t2[good]
ax2.hist(t2[good],normed=True,color='b',label='normal',alpha=.5)
t = t2[bad]
ax2.hist(t2[bad],normed=True,color='r',label='anormal',alpha=.5)
ax2.set_title('t2')

ax3 = fig.add_subplot(223)
ax3.hist(t12[good],normed=True,color='b',label='normal',alpha=.5)
ax3.hist(t12[bad],normed=True,color='r',label='anormal',alpha=.5)
ax3.set_title('t12')

ax4 = fig.add_subplot(224)
ax4.hist(t21[good],normed=True,color='b',label='normal',alpha=.5)
ax4.hist(t21[bad],normed=True,color='r',label='anormal',alpha=.5)
ax4.set_title('t21')

TypeError: list indices must be integers or slices, not Series

# Classification

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

C:\Users\benoit\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [28]:
features = np.column_stack((t1,t2,t12,t21,ps1,ps2,m12))
features.shape

(100, 7)

In [29]:
labels_=y = X_df.loc[:,"Normal"].values
labels_.shape

(100,)

In [30]:
#Logit
X_train, X_test, y_train, y_test = train_test_split(features, labels_, test_size=0.20)
logit = LogisticRegression()
result = logit.fit(X_train, y_train)
predictions = logit.predict(X_test)
print(accuracy_score(y_test, predictions))

0.7


In [33]:
#RandomForest
X_train, X_test, y_train, y_test = train_test_split(features, labels_, test_size=0.20)
RF =  RandomForestClassifier(n_estimators=150, max_depth=7, random_state=0)
result = RF.fit(X_train, y_train)
predictions = RF.predict(X_test)
print(accuracy_score(y_test, predictions))

0.7


In [32]:
#SVM
X_train, X_test, y_train, y_test = train_test_split(features, labels_, test_size=0.20)
clf = SVC()
result = clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))

0.7


In [81]:
rf = RandomForestClassifier()
svm = SVC()
print(cross_val_score(svm,features,labels_,cv=3))

[0.70588235 0.70588235 0.71875   ]


In [54]:
#RF
param_grid = {'n_estimators': [100, 150, 200, 250, 300],'max_depth': [2, 4, 6, 7]}
grid_RF = GridSearchCV(RF, param_grid, cv=10)
grid_RF.fit(X_train, y_train)
grid_RF. best_estimator_
grid_RF. best_params_

{'max_depth': 2, 'n_estimators': 100}

In [2]:
#Pour un SVM
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 1e-5, 1e-6],'C': [1, 10, 100, 1000]}]
print("# Tuning hyper-parameters for %s" % 'precision')
print()
clf = GridSearchCV(SVC(), tuned_parameters, cv=5)
clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()

SyntaxError: positional argument follows keyword argument (<ipython-input-2-02156ae22fd6>, line 5)

AttributeError: 'SVC' object has no attribute 'grid_scores_'